In [ ]:
from ratdata import data_manager as dm, process, ingest, plot
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import scipy.signal as signal
import peewee

In [ ]:
redo = False

lines = []
if redo:
    for i, r in enumerate(dm.RecordingFile.select().where(dm.RecordingFile.condition == 'baseline').order_by(peewee.fn.Random())):
        line = str.join('', (str(i), ': ', r.filename, '\n'))
        lines.append(line)
        filename = Path(r.dirname) / r.filename
        rec = ingest.read_mce_matlab_file(filename)
        if dm.is_recording_rejected(filename.name):
            continue

        fs = int(1/rec.dt)
        
        x = np.mean(rec.electrode_data, 0)
        if dm.is_recording_sliced(r.filename):
            slice = dm.RecordingSlice.select().join(dm.RecordingFile).where(dm.RecordingFile.filename == filename.name).get()
            start = int(slice.start * fs)
            end = int((slice.start + slice.length) * fs)
            x = x[start:end]
            
        f, psd = signal.welch(x, fs, nperseg=int(2*fs))
        m, b = process.fit_oof(f, psd, 2, 100)
        f[0] = 1e-20
        oof = f ** m * (np.e ** b)
        detrended = psd - oof
        idx = np.logical_and(f >= 10, f <= 30)
        plt.figure(figsize=(12, 6))
        plt.plot(f[idx], detrended[idx])
        plt.title('%03d' % i)
        # plt.ylim([-6e-5, 9e-5])
        plt.axhline(0, color='red')
        ax = plt.gca()
        ax.set_xticks(np.linspace(10, 30, 21), minor=False)
        plt.xlim([10, 30])
        plt.grid()
        plt.savefig('plots/blinded_power_spectra/baseline_%03d.png' % (i), facecolor='white', bbox_inches='tight')
        plt.close()

    with open('plots/blinded_power_spectra/key.txt', 'w') as f:
        f.writelines(lines)